<a href="https://colab.research.google.com/github/lakmaths/Data_lak/blob/main/hdb%20resale%20price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'resale-hdb-flat-prices-2000-2022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2199935%2F7388143%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20250220%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20250220T081740Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5edad8762e64b744c560a0e3f0b134c8aa4409780d77b29adc5a3c243e3b189ec1d4d7d4e2364e0afea30de39d5bb7a289bf623af2f8d7587ce2ce74fe04766626c0b0842fb481694e8974092429c785879f8ec9cdaa5d9b9ea12d7723a200aed22d30e7c72ac485d626b9e111934926b230ac7bba5dcaa47a6ede67bc757b08d47a26eb783c612a084ec17823a3304db8386607b001ae73bade88ad5878399d396442c76dc50c69f3f20c99d7230e12d47992373330ca62a8f984c901fb2e2fc5733bff1b1260dc89d9a57b836dc30495450c071b491977370e9867251ca33968654c58869aa8261f3467964de6530a7d6444ecce19d16cf0693120224cbf6c'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
# %% [code]
---
title: "Exploring Singapore HDB Resale Flat Price in 2012-2020"
author: "Syukri R"
date: '2022-05-08'
output: html_document
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```

#### 1. Preface

##### 1.1 About
Singapore is a little red dot country which experiences scarcity in the land plots for its growth and development. Fortunately, the Singapore government has adopted and mitigated the land scarcity mainly for settling the population through vertical residential replacing the kampong neighbourhood with Housing and Development Board (HDB) flats. Nowadays HDB flat is one of the basic necessities for every Singapore resident. HDB numbers have reached more than 1 million flats spread across 24 towns and 3 estates [link](https://www.hdb.gov.sg/about-us/our-role/public-housing-a-singapore-icon). The flats spell home to over 80% of Singapore's resident population.  A total of 3.04 million Singapore residents - or close to eight in 10 Singapore residents - lived in HDB flats in 2018 [link](https://www.straitstimes.com/singapore/singapore-resident-population-in-hdb-flats-falls-to-304m-with-smaller-households-spread). In 2021, a type of residencies in Singapore composed with HDB as 78,3%, condominiums and other apartments as 16,5, landed property as 4,9% and others type as 0,3 % [link](https://www.singstat.gov.sg/modules/infographics/population)
Singapore might be one of the top countries with the highest proportions of the population owning capital.

For close to 90 per cent of Singaporean households who own their homes, housing not only provides a roof over people’s heads but also works out to be one of the largest assets families hold over their lifetimes. Included in HDB flats that have intrinsic unique dual features of housing, both as a place to live in and a largely stable and appreciating investment, are mutually reinforcing in a country that continues to grow economically and attract people to come. During this period, the resale prices of HDB flats grew at 0.82 per cent per quarter on average, or an accumulated growth of nearly 70 per cent. Private landed, and non-landed residential property price indices have increased by 71.5 per cent and 58.0 per cent, respectively in this same time.[link](https://www.channelnewsasia.com/commentary/hdb-housing-and-development-board-housing-prices-real-estate-property-market-singapore-2339696)
For example how HDB flats become an investment asset can be seen through the sales of 4-room flats in Fernvale Grove which were bought in 2004 between S$129,00 and S$162,000 and had been transacting with an average transacted price of S$469,000 in 2021 – which is a 222 per cent return. The other transaction can be seen on HDB flats of Pinnacle@Duxton homes, the only batch of BTO flats built in the Central Area since 2004, buyers who bought from HDB in 2004 (at S$392,100 and S$451,500 for 4- and 5-rooms) and resold in 2021 (at the average resale transacted prices of S$851,868 and $1,001,289) saw their asset appreciate at 7.03 per cent each year over the 17 years – a pace that outstripped the STI Index’s 2.4 per cent for the same period. Despite the amazing growth of the resale price of HDB flats has been shown before, several areas have not enjoyed some magnitude like a resale in the Sengkang vicinity bought at the same time and resold in 2021 fetches about 69 per cent in contrast. [link](https://www.channelnewsasia.com/commentary/hdb-housing-and-development-board-housing-prices-real-estate-property-market-singapore-2339696)

Buying HDB flats in Singapore is a journey for Singapore residents. There are restrictions imposed since HDB flats received subsidies from the government. Resale HDB flats are more flexible in regulation compared with new HDB flats. The resale HDB flats market price is formed by various factors not only by supply and demand but also by the maturity of the location, completeness of supporting facilities, public amenities, distance from public transportation, schools and business centre, and government policies and the waiting time for new HDB flats. Since the pandemic hit construction work with the shortage in labour, this effect to the completion of new HDB flat projects in many areas which bring a longer lead time to 4 to 5 year [link](https://www.hdb.gov.sg/about-us/news-and-publications/publications/hdbspeaks/reduce-waiting-time-for-bto-flats). The condition mentioned above had driven for a rising sales price of resale HDB flats. When people can not queue in the waiting time the resale HDB flats market becomes an option for fulfilling their residential needed and their liquid investment for the future.

This case study tried to decipher a pattern in resale HDB flats from 2012 to 2022. Since the author does not has any background in economics and novice in R language, the analysis will touch the periphery and not dive deep inside the dataset which has been collected.

##### 1.2 Project Stakeholder
1. Singapore public who need information related to resale HDB flats from 2012 to 2022.
2. Singaporean/permanent residence/foreigner who wants to buy resale HDB flats and needs early information related to resale HDB flats prices from 2012 to 2022.
3. Anyone who has interested in resale HDB flats prices from 2012 to 2022.



#### 2. Objective

##### 2.1 Analysis Questions

1.  What are some patterns in resale HDB flats prices from 2012 to 2022?
2.  What is the location (a town and block/street) with a high number of transactions for resale HDB flats from 2012 to 2022?
3.  What is the note point that can be used for consideration in buying-selling resale HDB flats?

##### 2.2 Business Task

Exploring and identifying a pattern or trend of resale HDB flats in terms of informing Singaporean/permanent residents/other public who have interest in resale HDB flats.


#### 3. Setup environment
##### 3.1.Install and Loading Packages

```{r List of required packages, message=FALSE, warning=FALSE}
lib_List <- c("tidyverse","dplyr","ggplot2","ggrepel","tibble","lubridate","skimr","janitor","here","Rcpp","corrplot", "knitr", "rmarkdown")

instpack <- lib_List %in% installed.packages()[,"Package"]
if (any(instpack == FALSE)) {
  install.packages(lib_List[!instpack])
}

invisible(lapply(lib_List, library, character.only = TRUE))

```


##### 3.2 Importing Datasets

-   The dataset has been taken from Government official site. [link](<https://data.gov.sg/dataset/resale-flat-prices>).
-   The dataset will process and analysed with R.
-   Download Resale Flat Price dataset in Zip format and directly upload cloud \> projet.
-   Select all data in the Resale Flat Price dataset folder and imported it to Environment.

**Resale Flat Price Dataset**
```{r Importing the datasets, message=FALSE, warning=FALSE}
resale_prices_2000_2012 <-read_csv("../input/resale-hdb-flat-prices-2000-2022/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")

resale_prices_2012_2014 <-read_csv("../input/resale-hdb-flat-prices-2000-2022/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")

resale_prices_2015_2016 <-read_csv("../input/resale-hdb-flat-prices-2000-2022/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")

resale_prices_2017_onwards <-read_csv("../input/resale-hdb-flat-prices-2000-2022/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
```

##### 3.3. ROOC Dataset

Note point for dataset ROOC (Reliable, Original, Comprehensive and Cited).

**3.3.1. Reliable** (R) The dataset is highly reliable since it collected from official Singapore Government Data bureau link (<https://data.gov.sg/dataset/resale-flat-prices>). The dataset is borne available through licence <https://data.gov.sg/open-data-licence>, which is the dataset user should acknowledge the source of the datasets and include a link to the most recent, the datasets are provided on an "as is" and "as available" basis. The Agency (Singapore Government) makes no representations or warranties in relation to the datasets, including but not limited to any representation or warranty as to the accuracy, completeness, reliability, continued availability, timeliness, non-infringement, title, quality or fitness for any particular purpose of the datasets to the fullest extent permitted by the law.

**3.3.2 Original** (O) The data is directly collected from the first source of a government agency that has authority and is responsible for public residential in Singapore. The agency is dubbed as HDB (Housing Development Board).

**3.3.3 Comprehensive** (C) The metadata of these datasets automatically been included when user have downloaded the datasets. The metadata comprises the source of data, publisher, responsible person point of contact, last time updated, file format, frequently of updated and etc.

**3.3.4. Current** (C) The datasets is updated and the last update is on April 2022.

**3.3.5 Cited** (C) The datasets can be find in Data.gov.sg through link (<https://data.gov.sg/dataset/resale-flat-prices>)

##### 3.4 Cleaning The Datasets

###### 3.4.1 Checking Datasets Information
```{r Datasets info, message=FALSE, warning=FALSE, include=FALSE}
file.info(("resale-flat-prices/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv"), extra_cols = FALSE)

file.info(("resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv"), extra_cols = FALSE)

file.info(("resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv"), extra_cols = FALSE)

file.info(("resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv"), extra_cols = FALSE)
```

###### 3.4.2 Check Datasets Organization & Cleanliness
The trend analysis is only for 2012 - 2022, only a dataset from 2000 - 2022 will proceed.
```{r Datasets summary, message=FALSE, warning=FALSE}
skim(resale_prices_2000_2012)
summary(resale_prices_2000_2012)

skim(resale_prices_2012_2014)
summary(resale_prices_2012_2014)

skim(resale_prices_2015_2016)
summary(resale_prices_2015_2016)

skim(resale_prices_2017_onwards)
summary(resale_prices_2017_onwards)
```

###### 3.4.3 Removing Empty Columns and Rows
```{r Datasets cleaning, include=FALSE}
resale_prices_2000_2012 %>% remove_empty(which = c("rows", "cols"), quiet = FALSE)

resale_prices_2012_2014 %>% remove_empty(which = c("rows", "cols"), quiet = FALSE)

resale_prices_2015_2016 %>% remove_empty(which = c("rows", "cols"), quiet = FALSE)

resale_prices_2017_onwards %>% remove_empty(which =c("rows", "cols"), quiet = FALSE)
```

###### 3.4.4 Comparing Columns in All Datasets
```{r Datasets comparasion}
compare_df_cols(resale_prices_2000_2012,resale_prices_2012_2014,resale_prices_2015_2016,resale_prices_2017_onwards)
```

##### 3.5 Note for The Datasets
Since the datasets have been pulled from official government agencies, there is a high possibility of not finding any irregular data, errors, data incompletion, irregular naming, and any variables associated with dirty data. From several functions that have been run before, there are some differences in the number of columns from datasets 2002-2012, 2012-2014 compared with datasets 2015-2016 and 2017-onwards. In the first group, there are 10 columns, whereas the rest of the two datasets have 11 columns.

The difference is in the "remaining_lease" column. The other differences is a type of the "remaining_lease" column in 2017_onwards which a character column. The next procedure will add "remaining_lease" columns in two early datasets, combine all datasets to become a single dataset, and fix the dataset period from 2012 to 2022 (onwards). Befor merging all datasets, we have to convert the "remaining_lease" column in 2017_onwards from character type to numeric type column.

##### 3.6 Processing The Datasets
###### 3.6.1 Combining Dataset, Bind resale_prices_2000_2012 with resale_prices_2012_2014
```{r Merging datasets}
new_resale_prices_2012_2014 <- bind_rows(resale_prices_2000_2012, resale_prices_2012_2014)
```

###### 3.6.2 Adding New "remaining_lease" Columns to 2012_2014 Dataset
```{r Dataset modification}
new_resale_prices_2012_2014 ["remaining_lease"] <- (99 - (2022 - new_resale_prices_2012_2014$lease_commence_date))
nrow(new_resale_prices_2012_2014)
```

###### 3.6.3 Deleting All Observation Column Below 2012
```{r Dataset adjustment}
new_2_resale_prices_2012_2014 <- subset(new_resale_prices_2012_2014, month<2012)
nrow(new_2_resale_prices_2012_2014)
```

###### 3.6.4 Creating New Datasets for Resale Prices 2012-2014
```{r Binding new datasets}
nrow(new_resale_prices_2012_2014) - nrow(new_2_resale_prices_2012_2014)
new_3_resale_prices_2012_2014 <- new_resale_prices_2012_2014[366464:421854, ]
nrow(new_3_resale_prices_2012_2014)
```

###### 3.6.5 Modified The "remaining_lease" Column in 2017_onwards
```{r Modification for new dataset}
new_2_resale_prices_2017_onwards <- subset(resale_prices_2017_onwards, select = - remaining_lease)
new_2_resale_prices_2017_onwards ["remaining_lease"]<- (99 - (2022 - new_2_resale_prices_2017_onwards$lease_commence_date))
```

###### 3.6.6 Merging All Datasets From 2012-onwards
```{r All new dataset}
all_resale_prices_2012_onwards <- new_3_resale_prices_2012_2014 %>%
  bind_rows(resale_prices_2015_2016, new_2_resale_prices_2017_onwards)

skim_without_charts(all_resale_prices_2012_onwards)
summary(all_resale_prices_2012_onwards)
```

###### 3.6.7 Splitting Month/Year Become Year and Month Columns
```{r Adjusting column}
all_resale_prices_2012_2022 <- rename( all_resale_prices_2012_onwards, "month_year" = "month")

resale_prices_2012_2022<- all_resale_prices_2012_2022 %>% separate(month_year, c('year', 'month'))

skim(resale_prices_2012_2022)
summary(resale_prices_2012_2022)
```




#### 4. Analyze The Dataset
##### 4.1. Identified The Patterns
###### 4.1.1 Selling Flat Distribution Base on Flat Types
```{r Represent percentages resale sales flat 2012-2022, message=FALSE, warning=FALSE}
#0
frequent_of_selling_unit_02 <- resale_prices_2012_2022 %>% count(flat_type, sort = TRUE)

frequent_of_selling_unit_02 <- frequent_of_selling_unit_02 %>% rename(number_unit = n)

frequent_of_selling_unit_02["unit_percentage"] <- (frequent_of_selling_unit_02$number_unit/sum(frequent_of_selling_unit_02$number_unit)*100)


piechart_df <- frequent_of_selling_unit_02
piechart_df2 <- piechart_df %>% mutate(csum = rev(cumsum(rev(unit_percentage))), pos = unit_percentage/2 + lead(csum, 1), pos = if_else(is.na(pos), unit_percentage/2, pos))


ggplot(piechart_df, aes(x = "" , y = unit_percentage, fill = fct_inorder(flat_type))) +
  geom_col(width = 1, color = 1) +
  coord_polar(theta = "y") +
  scale_fill_brewer(palette = "Pastel1") +
  geom_label_repel(data = piechart_df2, aes(y = pos, label = paste0(round(unit_percentage,3), "%")), size = 4.5, nudge_x = 1, show.legend = FALSE) +
  guides(fill = guide_legend(title = "Flat Type"), reverse = FALSE) +
  labs(title="Resale Flat Type Distribution in 2012 - 2022") +
  theme_void()
```

```{r The most selling flat types, warning=FALSE}
#1
ggplot(resale_prices_2012_2022, aes(x = flat_type, fill = year)) +
  geom_bar() +
  labs(title="The Most Selling Flat Types in 2012 - 2022") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
```

```{r Number of Selling Flat Base on Flat Types, warning=FALSE}
#2
number_selling_by_flat_type <-resale_prices_2012_2022 %>% count(flat_type)
number_selling_by_flat_type<-rename(number_selling_by_flat_type, "units" = "n")
number_selling_by_flat_type %>%add_row(flat_type = "Total", units = sum(.$units))
```

###### 4.1.2 Trends by Year From 2012 to 2022
```{r Flat types sales distribution, warning=FALSE}
#0
ggplot(resale_prices_2012_2022, aes(x = year, fill = flat_type)) +
  geom_bar() +
  labs(title="The Year with Highest Sales Rate in 2012 - 2022 ") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r The Selling Flat Number by Year, warning=FALSE}
#1
number_flat_by_year <- resale_prices_2012_2022 %>% count(year, sort = TRUE)
number_flat_by_year<-rename(number_flat_by_year, "units" = "n")
number_flat_by_year %>%add_row(year = "Total", units = sum(.$units))
```

###### 4.1.3 Resale Price Trends From 2012 to 2022

```{r Convert year character become year numeric, message=FALSE, warning=FALSE}
#0
resale_prices_2012_2022_N <-resale_prices_2012_2022
resale_prices_2012_2022_N$year <- as.numeric(as.character(resale_prices_2012_2022_N$year))

pvt_resale_price_2012_2022 <-resale_prices_2012_2022_N%>%
  group_by (flat_type) %>%
  summarize(total_sales = n(),
            mean_sales_price = mean(resale_price),
            mean_floor_area = mean(floor_area_sqm),
            mean_remain_lease = mean(remaining_lease))
print(pvt_resale_price_2012_2022)
```


```{r finding mean for sales price, message=FALSE, warning=FALSE}
#1
pvt_resale_price_2012_2022_wide <-resale_prices_2012_2022_N %>%
  group_by (flat_type, year) %>%
  summarize(total_sales = n(),
            mean_sales_price = mean(resale_price),
            mean_floor_area = mean(floor_area_sqm),
            mean_remain_lease = mean(remaining_lease),
            max_sales_price = max(resale_price),
            min_sales_price = min(resale_price))

print(pvt_resale_price_2012_2022_wide)
```

###### 4.1.3.a  Mean Resale Price Trends From 2012 to 2022

```{r Ploting mean sales price from 2012 to 2022, message=FALSE, warning=FALSE}
#0
pvt_resale_price_2012_2022_wide %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

###### 4.1.3.b  Detail Mean Resale Price Trends From 2012 to 2022

```{r Mean of resale prices 1 ROOM, message=FALSE, warning=FALSE}
#0
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on 1 Room Flat Type", caption = "Mean Resale Price Trend on 1 Room Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices 2 ROOM, message=FALSE, warning=FALSE}
#1
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on 2 Room Flat Type", caption = "Mean Resale Price Trend on 2 Room Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices 3 ROOM, message=FALSE, warning=FALSE}
#2
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on 3 Room Flat Type",  caption = "Mean Resale Price Trend on 3 Room Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices 4 ROOM, message=FALSE, warning=FALSE}
#3
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on 4 Room Flat Type",caption = "Mean Resale Price Trend on 4 Room Flat Type"  ) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices 5 ROOM, message=FALSE, warning=FALSE}
#4
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on 5 Room Flat Type", caption = "Mean Resale Price Trend on 5 Room Flat Type" ) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices EXECUTiVE, message=FALSE, warning=FALSE}
#5
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on EXECUTIVE Flat Type" ,caption = "Mean Resale Price Trend on EXECUTIVE Room Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Mean of resale prices MULTI-GENERATION, message=FALSE, warning=FALSE}
#6
pvt_resale_price_2012_2022_wide %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(year, mean_sales_price, colour = flat_type))+
  geom_point () +
  geom_smooth() +
  geom_line()+
  facet_wrap(~flat_type)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  labs(tittle = "Mean Resale Price Trend on Multi-Generation Flat Type",caption = "Mean Resale Price Trend on MULTI-GENERATION Type" ) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

###### 4.1.3.c  Plot for Resale Price Trends From 2012 to 2022

```{r Trend for total sales flat type, message=FALSE, warning=FALSE}
#0
resale_prices_2012_2022 %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_type)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend in 2012-2022 by All Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for 1 ROOM-2012_2022, message=FALSE, warning=FALSE}
#1
resale_prices_2012_2022 %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on 1 ROOM Flat Type" ) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for 2 ROOM-2012_2022, message=FALSE, warning=FALSE}
#2
resale_prices_2012_2022 %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on 2 ROOM Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for 3 ROOM-2012_2022, message=FALSE, warning=FALSE}
#3
resale_prices_2012_2022 %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on 3 ROOM Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for 4 ROOM-2012_2022, message=FALSE, warning=FALSE}
#4
resale_prices_2012_2022 %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on 4 ROOM Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for 5 ROOM-2012_2022, message=FALSE, warning=FALSE}
#5
resale_prices_2012_2022 %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on 5 ROOM Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for EXECUTIVE-2012_2022, message=FALSE, warning=FALSE}
#6
resale_prices_2012_2022 %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on EXECUTIVE Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r Price trend & volume for MULTI-Generation-2012_2022, message=FALSE, warning=FALSE}
#7
resale_prices_2012_2022 %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = year, y = resale_price, colour = flat_model)) +
  geom_point( size = 5, alpha = 0.3) +
  geom_line(size = 1) +
  geom_smooth() +
  theme_minimal() +
  labs(caption = "Resale Price Trend by Years on MULTI-GENERATION Flat Type") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

###### 4.1.4 The Town with The Highest Number of Resale HDB Flats Sales
```{r The town with high selling flat, message=FALSE, warning=FALSE}
#0
ggplot(resale_prices_2012_2022, aes(x = town, fill = flat_type)) +
  geom_bar() +
  labs(title="The Town with High Selling Flat Units in 2012-2022") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
```

```{r The town with highest selling units, message=FALSE, warning=FALSE}
#1
number_flat_by_town <- resale_prices_2012_2022 %>% count(town, sort = TRUE)
number_flat_by_town<-rename(number_flat_by_town, "units" = "n")
number_flat_by_town %>%add_row(town = "Total", units = sum(.$units))
```

```{r The street with highest selling units, message=FALSE, warning=FALSE}
#2
number_flat_by_street <- resale_prices_2012_2022 %>% count(street_name, sort = TRUE)
number_flat_by_street <-rename(number_flat_by_street , "units" = "n")
number_flat_by_street  %>%add_row(street_name = "Total", units = sum(.$units))
```

###### 4.1.5 Resale Price Distribution and Storey Range in 2012 - 2022

```{r Total sales with storey range, message=FALSE, warning=FALSE}
#0
ggplot(resale_prices_2012_2022, aes(x = storey_range, y = resale_price, color = flat_type)) +
  geom_point() +
  geom_smooth() +
  labs(title="Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

```

```{r message=FALSE, warning=FALSE}
#1
number_selling_by_storey <- resale_prices_2012_2022 %>% count(storey_range, sort = TRUE)
number_selling_by_storey <-rename(number_selling_by_storey, "units" = "n")
number_selling_by_storey  %>%add_row(storey_range = "Total", units = sum(.$units))
```



```{r 1 ROOM vs storey range, message=FALSE, warning=FALSE}
#2
resale_prices_2012_2022 %>%
  filter (flat_type == "1 ROOM") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="1 ROOM Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
```

```{r 2 ROOM vs storey range, message=FALSE, warning=FALSE}
#3
resale_prices_2012_2022 %>%
  filter (flat_type == "2 ROOM") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="2 ROOM Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

```

```{r 3 ROOM vs storey range, message=FALSE, warning=FALSE}
#4
resale_prices_2012_2022 %>%
  filter (flat_type == "3 ROOM") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="3 ROOM Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
```

```{r 4 ROOM vs storey range, message=FALSE, warning=FALSE}
#5
resale_prices_2012_2022 %>%
  filter (flat_type == "4 ROOM") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="4 ROOM Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

```

```{r 5 ROOM vs storey range, message=FALSE, warning=FALSE}
#6
resale_prices_2012_2022 %>%
  filter (flat_type == "5 ROOM") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="5 ROOM Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

```

```{r EXECUTIVE vs storey range, message=FALSE, warning=FALSE}
#7
resale_prices_2012_2022 %>%
  filter (flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="EXECUTIVE Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

```

```{r MULTI-GENERATION vs storey range, message=FALSE, warning=FALSE}
#8
resale_prices_2012_2022 %>%
  filter (flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = storey_range, y = resale_price, color = flat_model)) +
  geom_point() +
  geom_smooth() +
  geom_line() +
  labs(title="MULTI-GENERATION Resale Price Distribution by Storey Range in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
```

###### 4.1.6 Resale Price Distribution and Remaining Lease in 2012 - 2022

```{r Remaining lease, message=FALSE, warning=FALSE}
#0
ggplot(resale_prices_2012_2022, aes(x = remaining_lease, fill = flat_type)) +
  geom_bar() +
  labs(title="The Most Selling Units by Remaining Lease in 2012-2022") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r}
#1
number_flat_remain_lease <- resale_prices_2012_2022 %>% count(remaining_lease, sort = TRUE)
number_flat_remain_lease <- rename(number_flat_remain_lease , "units" = "n")
number_flat_remain_lease$remaining_lease <- as.character(as.numeric(number_flat_remain_lease$remaining_lease))
number_flat_remain_lease  %>% add_row(remaining_lease = "Total", units = sum(.$units))
```


```{r message=FALSE, warning=FALSE}
#1 1 ROOM Trend

resale_prices_2012_2022 %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="1 ROOM Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="1 ROOM Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#2 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="2 ROOM Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="2 ROOM Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#3 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="3 ROOM Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="3 ROOM Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#4 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="4 ROOM Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="4 ROOM Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#5 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="5 ROOM Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="5 ROOM Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#EXECUTIVE Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="EXECUTIVE Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="EXECUTIVE Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#MULTI-GENERATION Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="MULTI-GENERATION Units Sales Trend by Remaining Lease") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
resale_prices_2012_2022 %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = remaining_lease, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="MULTI-GENERATION Units Sales Trend by Remaining Lease") +
  facet_wrap(~ year) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))

```


###### 4.1.7 Resale Price Distribution with Floor Area in 2012 - 2022

```{r Resale Price with floor area sqm, message=FALSE, warning=FALSE}
#1
ggplot(resale_prices_2012_2022, aes(x = floor_area_sqm, y = resale_price, color = flat_type)) +
  geom_point() +
  geom_smooth() +
  labs(title="Resale Price Trend by Floor Area Sqm in 2012 - 2022 ") +
  scale_y_continuous(labels = function(y) format(y, scientific = FALSE)) +
  theme(axis.text.x = element_text(angle = 45, vjust = 0.5, hjust=1))
```

```{r message=FALSE, warning=FALSE}
#1 ROOM
resale_prices_2012_2022 %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="1 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#1 ROOM
resale_prices_2012_2022 %>%
  filter(flat_type == "1 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~year) +
  labs(title="1 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#2 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="2 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

```{r message=FALSE, warning=FALSE}
#2 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "2 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~year) +
  labs(title="2 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#3 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="3 ROOM Units Sales Trend by Floor Areae") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#3 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "3 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~ year) +
  labs(title="3 ROOM Units Sales Trend by Floor Areae") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#4 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="4 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#4 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "4 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~ year) +
  labs(title="4 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#5 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="5 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#5 ROOM Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "5 ROOM") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~year)+
  labs(title="5 ROOM Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```



```{r message=FALSE, warning=FALSE}
#EXECUTIVE Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="EXECUTIVE Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#EXECUTIVE Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "EXECUTIVE") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~ year) +
  labs(title="EXECUTIVE Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#MULTI-GENERATION Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  labs(title="MULTI-GENERATION Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```


```{r message=FALSE, warning=FALSE}
#MULTI-GENERATION Trend
resale_prices_2012_2022 %>%
  filter(flat_type == "MULTI-GENERATION") %>%
  ggplot(aes(x = floor_area_sqm, y = resale_price, colour = flat_model)) +
  geom_point() +
  geom_line() +
  geom_smooth() +
  facet_wrap(~ year)+
  labs(title="MULTI-GENERATION Units Sales Trend by Floor Area") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))
```

#### 5. Founding and Insight
##### 5.1. Founding From The Analysis

Between 2012 to April 2022 there are 218.008 total transactions on resale HDB flats. The resale HDB flats transaction consists of 7 types of HDB flats. The 7 types are 1 room, 2 room, 3 room, 4 room, 5 room, Executive and Multi-Generation flat type. The sales percentage of each type is as follow 1 room with 0.049% (107 units), 2 room with 1,288% (2807 units),  3 room with 25.501% (55595 units), 4 room  with 40.768% (88878 units), 5 room with 24.445% (53291 units), Executive with 7.908% (17239 units) and Multi-Generation with 0.042% (91 units) .

During the period the highest transaction number happened in 2021 with 29,091 resale HDB flats transacted. Meanwhile, the lowest transaction happened in 2014 with 16096 resale HDB flats transacted.

Through inspecting the mean of prices in the period here are several points have founded.

1. The resale prices for 1, 2, and 3 room flats from 2012 until 2019 were going down and started rebounding over the next years.

2. The resale prices for 4,5 room and an Executive flats from 2012 to 2013 have risen and started going down in 2014. The prices slid until 2019 and made rebounded over the next year.

3. The Multi-Generation resale flat has a different pattern from all the flats mentioned before. It is more dynamic where the prices have a three-time low in 2014, 2016 and 2020. The price has stabilized in  2022 and probably reach the same level as in 2012.


From 28 towns and estates, the dataset has shown 10 towns with high resale HDB flats transacted. The town is

1. Sengkang  (16543 units),

2. Jurong West (15968),

3. Woodlands (15929 units),

4. Tampines (15222 units),

5. Yishun (4337 units),

6. Bedok (12904 units),

7. Punggol (12252 units),

8. Hougang (10914 units),

9. Ang Mo Kio   (9843 units),

10. Choa Chu Kang (9619 units).


The dataset also has shown from 561 streets (block) with 10 streets (block) that have the biggest transaction. The streets (block) are;

1. Yishun Ring Rd (3400 units),

2. Bedok Reservoir Rd (2656 units),

3. Ang Mo Kio Ave 10 (2434 units),

4. Ang Mo Kio 3 (2213),

5. Punggol Field (1998 units),

6. Punggol Dr (1901 units),

7. Punggol Ctrl (1759 units),

8. Hougang Ave (1717 units),

9. Fernvale Rd (1683 units),

10. Choa Chu Kang Cres (1594 units).


In the storey range exploration, there are founded which storey range with highest resale HDB flats transacted. Here are the 10 favourite storey ranges:

1. 04 TO 06 (50171 units),

2. 07 TO 09 (44970 units),

3. 01 TO 03 (39735 units),

4. 10 TO 12 (39450 units),

5. 13 TO 15 (18723 units),

6. 16 TO 18 (8365 units),

7. 19 TO 21 (3540 units),

8. 01 TO 05 (2700 units),

9. 22 TO 24 (2506 units),

10. 06 TO 10 (2474 units).


In the remaining lease range, there are 10 sorts of  remaining lease year have been found as follow;

1. 62 years (13271 units),

2. 61 years (9928 units),

3. 92 years (8173 units),

4. 65 years (7953 units),

5. 78 years (7302 units),

6. 80 years (7015 units),

7. 64 years (6748 units),

8. 63 years (6455 units),

9. 76 years (6261 units),

10. 75 years (6159 units).


##### 5.2. Insight

1. Resale HDB flat price is very dynamic and can be influenced by many factors. However, the government's role in price movement is one dominant factor. For instance, HDB has a valuation policy for resale HDB flats which has the intention for keep the prices afloat, not dipe down or rise sky-high. In other mean HDB has tried to hold the flats not to have unlikely depreciation. It will be a bad precedent for the HDB dwelling model if have depreciation which a lot of conscious consumers would go for renting the flat rather than buying it.

2. Besides the resale HDB flat prices can go up, it also can go down. This can be seen in the year 2019 mostly mean of resale HDB flat price slip down. The factors which drove prices down were a big supply of HDB resale flats (about 25,000 BTO flats will reach the minimum occupation period of five years this year/2019 - when the first owner can sell their HDB flats),  23,000 BTO flats completed in 2019, concerns over depleting leases on HDB flats, an update on housing policies such as the Voluntary En bloc Redevelopment Scheme and Home Improvement Programme II (announced by Prime Minister Lee Hsien Loong at last year’s National Day Rally), the impact of July 2018’s cooling measures on the private residential market. [link](https://www.todayonline.com/singapore/hdb-resale-prices-drop-sixth-straight-year-declining-09-2018)

3. Location, location and location. This word is always chanted when considering buying a property. In resale HDB flats, the most active flats transacted are in the suburbs areas or the town where has more distance to business centre district. Namely, the 5 top towns are Sengkang, Jurong West, Woodlands, Tampines and Yishun. When considering buying resale HDB flats as an investment rather than for a long time dwelling place, buying resale HDB flats in a suburb town can be one main consideration.

4. To obtain resale HDB flats with the long remaining lease is one objective in buying the flats. However, the fact has been found is speak in other ways. The most active resale HDB flats are in the range of 60 until 80 years remaining lease. It is quite interesting where resale activities are dominated by the flats that have lost one of third of their life span. As a common assumption, the number of resale activities is going down for the flats with 40 to 50 years remaining lease.

5. On many occasions a lot of assumptions said the flats in higher levels tend to be more attractive which makes them more active in resale activity and tend to pricy. This assumption can be disputed since what analysis found, that the most selling resale HDB flats are in lower and middle floor areas (1 to 10). For low, middle and high floors unit categorization can be found through this [link](https://singaporepublichousing.fandom.com/wiki/HDB_unit_Elevation-Ground_floor,low_floor,mid-floor_and_high_floor#Low_floor_units).

6. In the analysis some patterns had found in the resale HDB flat prices;

a. 1 room flat, nowadays price means tend to below 2012 mean price.

b. 2 room flat has slightly increment in a mean of prices compare with 2012 mean of price.

c. 3, 4, 5 room and Executive flat can be said in "bullish" state where their price has gone skyrocket comparing 2012 mean of price, even also compared with 2019 mean of price.

d. Multi-Generation flat means of price tend to be stable where nowadays mean of price slightly rise to compare on 2012 mean of price.

7. Final word, buying a resale HDB flat is not a simple process there are many considerations for realizing it. Since the resale HDB flat price is higher than a new Flat price which is heavily subsidised by the government. The need for dwelling and investment has been formed in the resale HDB flat. Opting for less pricy resale HDB flat can be chosen whenever is not in a prime location, less remaining lease not to consider as unattractive factors. Again government policy is the key in weighing the resale HDB flats. When HDB valuation is more tends to price fencing which keeps the HDB price afloat.






















